In [47]:
import pandas as pd
import regex as re
import numpy as np
from gensim.models import word2vec
from sklearn.model_selection import train_test_split
import string
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Bidirectional , LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import gensim.downloader as api
import nltk
from nltk import PorterStemmer , word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
nltk.download("wordnet")
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Load the CSV file into a pandas DataFrame
file_path = "/content/drive/MyDrive/Data-PF/spam.csv"
df = pd.read_csv(file_path, encoding='latin1', usecols=['v1', 'v2'])

# Display the first few rows
print(df.head())

In [5]:
# changing labels

df.columns =['label', 'email']
# Assuming 'ham' is encoded as 0 and 'spam' is encoded as 1
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

df.head()

,label,email
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
def preprocess(Email):
    # Remove punctuation
    Email = re.sub(r'[^\w\s]', '', Email)

    # Convert to lowercase
    Email = Email.lower()

    # Tokenize words and remove stopwords
    stop_words = set(stopwords.words("english"))
    Stemmer = PorterStemmer()
    words = word_tokenize(Email)
    filtered_words = [Stemmer.stem(word) for word in words if word not in stop_words]
    preprocessed_email = " ".join(filtered_words)

    return preprocessed_email

In [ ]:
#Getting datasets using bow and embed

df['processed_email'] = df['email'].apply(preprocess)

print(df['processed_email'])


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['processed_email'])
tokenizer.word_counts

In [ ]:
sequences = tokenizer.texts_to_sequences(df['processed_email'])
sequences

In [45]:
pad_df =pad_sequences(sequences, padding="post")
pad_df.shape
print(pad_df)

[[   5 3753  289 ...    0    0    0]
 [  11  250  561 ...    0    0    0]
 [  13  378    3 ...    0    0    0]
 ...
 [8092 1065 8093 ...    0    0    0]
 [ 147 1335 1745 ...    0    0    0]
 [2227  403  179 ...    0    0    0]]


In [38]:
X= np.array(pad_df)
Y= df['label']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y , test_size=0.2, random_state=42)
print( X_train.shape, Y_train.shape)

(4457, 80) (4457,)


In [43]:
embedding = np.array(pad_df)

In [46]:
# SImple RNN model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=80, input_length= 80) )
model.add(SimpleRNN(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, Y_test)
print("Test Accuracy:", accuracy)

Epoch 1/10
112/112 [==============================] - 6s 32ms/step - loss: 0.4354 - accuracy: 0.8609 - val_loss: 0.4035 - val_accuracy: 0.8621
Epoch 2/10
112/112 [==============================] - 3s 29ms/step - loss: 0.3930 - accuracy: 0.8670 - val_loss: 0.4108 - val_accuracy: 0.8621
Epoch 3/10
112/112 [==============================] - 4s 33ms/step - loss: 0.3942 - accuracy: 0.8670 - val_loss: 0.4050 - val_accuracy: 0.8621
Epoch 4/10
112/112 [==============================] - 5s 42ms/step - loss: 0.3950 - accuracy: 0.8670 - val_loss: 0.4018 - val_accuracy: 0.8621
Epoch 5/10
112/112 [==============================] - 3s 31ms/step - loss: 0.3939 - accuracy: 0.8670 - val_loss: 0.4043 - val_accuracy: 0.8621
Epoch 6/10
112/112 [==============================] - 3s 30ms/step - loss: 0.3951 - accuracy: 0.8670 - val_loss: 0.4013 - val_accuracy: 0.8621
Epoch 7/10
112/112 [==============================] - 4s 35ms/step - loss: 0.3949 - accuracy: 0.8670 - val_loss: 0.4020 - val_accuracy: 0.8621

In [48]:
# Bidriect RNN model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=80, input_length= 80) )
model.add(Bidirectional(SimpleRNN(64, activation='relu')))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, Y_test)
print("Test Accuracy:", accuracy)

Epoch 1/10
112/112 [==============================] - 9s 55ms/step - loss: 0.2777 - accuracy: 0.8825 - val_loss: 0.2451 - val_accuracy: 0.9462
Epoch 2/10
112/112 [==============================] - 6s 51ms/step - loss: 0.0456 - accuracy: 0.9882 - val_loss: 0.1163 - val_accuracy: 0.9753
Epoch 3/10
112/112 [==============================] - 7s 61ms/step - loss: 0.0277 - accuracy: 0.9950 - val_loss: 0.0817 - val_accuracy: 0.9787
Epoch 4/10
112/112 [==============================] - 5s 48ms/step - loss: 0.0072 - accuracy: 0.9983 - val_loss: 0.7700 - val_accuracy: 0.9809
Epoch 5/10
112/112 [==============================] - 7s 64ms/step - loss: 0.0018 - accuracy: 0.9997 - val_loss: 0.4433 - val_accuracy: 0.9821
Epoch 6/10
112/112 [==============================] - 5s 47ms/step - loss: 0.0026 - accuracy: 0.9994 - val_loss: 0.9218 - val_accuracy: 0.9798
Epoch 7/10
112/112 [==============================] - 9s 77ms/step - loss: 4.3597e-04 - accuracy: 1.0000 - val_loss: 1.8007 - val_accuracy: 0.

In [49]:
# Bidriect RNN model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=80, input_length= 80) )
model.add(Bidirectional(LSTM(64, activation='relu')))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, Y_test)
print("Test Accuracy:", accuracy)

Epoch 1/10
112/112 [==============================] - 20s 156ms/step - loss: 0.6824 - accuracy: 0.8670 - val_loss: 0.3149 - val_accuracy: 0.8621
Epoch 2/10
112/112 [==============================] - 13s 117ms/step - loss: 34860964.0000 - accuracy: 0.9035 - val_loss: 0.1084 - val_accuracy: 0.9664
Epoch 3/10
112/112 [==============================] - 13s 111ms/step - loss: 0.0581 - accuracy: 0.9806 - val_loss: 2238.1321 - val_accuracy: 0.9753
Epoch 4/10
112/112 [==============================] - 13s 118ms/step - loss: nan - accuracy: 0.9355 - val_loss: nan - val_accuracy: 0.8621
Epoch 5/10
112/112 [==============================] - 14s 122ms/step - loss: nan - accuracy: 0.8670 - val_loss: nan - val_accuracy: 0.8621
Epoch 6/10
112/112 [==============================] - 13s 119ms/step - loss: nan - accuracy: 0.8670 - val_loss: nan - val_accuracy: 0.8621
Epoch 7/10
112/112 [==============================] - 13s 119ms/step - loss: nan - accuracy: 0.8670 - val_loss: nan - val_accuracy: 0.8621